In [1]:
from sklearn import tree
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

In [9]:
data = pd.read_csv("/Users/James/Desktop/countries.csv",usecols=[i for i in range(1,12)])
data

,Country,Quality of Life,Purchasing Power,Safety,Healthcare,Cost of Living,Property Price to Income,Traffic Commute Time,Pollution,Climate,Happiness
0,Switzerland,188.36,102.77,78.38,74.08,125.02,8.18,28.67,19.86,80.05,7.560
1,Denmark,186.25,86.43,73.78,79.79,88.53,7.00,28.69,21.29,81.80,7.646
2,Netherlands,180.27,76.65,72.84,75.28,78.93,7.28,27.64,25.39,87.00,7.449
3,Finland,178.95,80.11,72.41,76.19,76.35,8.57,28.91,11.99,56.64,7.809
4,Australia,178.41,91.07,56.97,77.86,80.75,7.27,34.81,23.69,92.70,7.223
...,...,...,...,...,...,...,...,...,...,...,...
74,Kenya,76.92,25.37,39.86,57.53,35.82,27.05,56.39,75.92,99.79,4.583
75,Sri Lanka,75.52,18.43,58.61,72.72,30.41,38.56,56.74,59.97,59.11,4.327
76,Bangladesh,64.54,21.76,36.10,42.22,33.01,14.91,57.12,84.94,72.91,4.833
77,Iran,63.60,16.82,50.62,52.33,40.64,34.87,47.25,75.70,70.99,4.672
